# **Skin Lesion Analyzer with Deep Learning**



### **Outline**
Use these links to jump to specific sections of this project.

1. Import Packages
2. Load and Transform the Dataset
3. Model Development
4. Model Training 
5. Prediction and Evaluation

### 1. Import Packages

In [5]:
import os
import glob
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import DenseNet121

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 2. Load and Transform the Dataset

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE


class DatasetBuilder():
    def __init__(self, base_dir, csv_file):
        self.base_dir = base_dir
        self.csv_file = csv_file
        
    def transform_df(self, base_dir, csv_file):
        df = pd.read_csv(csv_file)
        image_path_dict = {os.path.splitext(os.path.basename(x))[0]: x 
                       for x in glob.glob(os.path.join(base_dir, '*', '*.jpg'))}
        label_dict = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3,
                      'mel': 4, 'nv': 5, 'vasc': 6}
        df['image_path'] = df['image_id'].map(image_path_dict.get)
        df['label_id'] = df['dx'].map(label_dict.get)
        return df
    
    def split_df(self):
        dataframe = self.transform_df(self.base_dir, self.csv_file)
        train_df, val_df = train_test_split(dataframe, test_size=0.15)
        train_df, test_df = train_test_split(train_df, test_size=0.10)
        return train_df, val_df, test_df
    
    def get_labels(self, dataframe):
        label_list = dataframe.label_id.values
        labels = to_categorical(label_list, num_classes=7)
        return labels
    
    def decode_image(self, filename, label=None, image_size=(224, 224)):
        bits = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(bits, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, image_size)
        return image, label
    
    def input_fn(self, dataframe, batch_size=32, mode=None):
        image_list = dataframe.image_path.values
        labels = self.get_labels(dataframe)
        ds = (tf.data.Dataset     
                .from_tensor_slices((image_list, labels))
                .map(self.decode_image, num_parallel_calls=AUTOTUNE)
                .cache()
                .repeat()
                .shuffle(buffer_size = 10 * batch_size)
                .batch(batch_size)
                .prefetch(AUTOTUNE))
        return ds

    def create_dataset(self):
        train_df, val_df, test_df = self.split_df()
        train_ds = self.input_fn(train_df)
        val_ds = self.input_fn(val_df)
        return train_ds, val_ds

### 3. Model Development

In [12]:
image_shape = (224, 224, 3)
base_learning_rate = 0.0001

base_model = DenseNet121(input_shape=image_shape, 
                         include_top=False,
                         weights='imagenet')
base_model.trainable = True

image_input = keras.Input(shape=image_shape)
x = base_model(image_input)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(7, activation='sigmoid')(x)
model = keras.Model(inputs=image_input, outputs=x)

model.compile(optimizer=keras.optimizers.Adam(lr=base_learning_rate/10),
                loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

### 4. Model Training

In [13]:
def train_and_evaluate(params):
    csv_file = params['csv_file']
    base_dir = params['base_dir']
    batch_size = params['batch_size']
    num_epochs = params['num_epochs']
    train_steps = params['train_steps']
    val_steps = params['val_steps']

    builder = DatasetBuilder(base_dir, csv_file)
    train_ds, val_ds = builder.create_dataset()
    
    history = model.fit(train_ds,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=val_ds,
                        steps_per_epoch=train_steps,
                        validation_steps=val_steps)
    return history

In [14]:
params = {
    'csv_file': '../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv',
    'base_dir': '../input/skin-cancer-mnist-ham10000',
    'batch_size': 32,
    'num_epochs': 10,
    'train_steps': 239,
    'val_steps': 46
}

In [15]:
train_and_evaluate(params)

Epoch 1/10
239/239 [==============================] - 99s 413ms/step - loss: 1.6960 - accuracy: 0.4799 - val_loss: 1.6371 - val_accuracy: 0.6698
Epoch 2/10
239/239 [==============================] - 88s 368ms/step - loss: 1.4594 - accuracy: 0.7257 - val_loss: 1.4526 - val_accuracy: 0.7609
Epoch 3/10
239/239 [==============================] - 88s 369ms/step - loss: 1.4012 - accuracy: 0.7690 - val_loss: 1.3835 - val_accuracy: 0.7908
Epoch 4/10
239/239 [==============================] - 88s 368ms/step - loss: 1.3645 - accuracy: 0.7948 - val_loss: 1.3551 - val_accuracy: 0.8043
Epoch 5/10
239/239 [==============================] - 88s 368ms/step - loss: 1.3384 - accuracy: 0.8227 - val_loss: 1.3419 - val_accuracy: 0.8077
Epoch 6/10
239/239 [==============================] - 88s 368ms/step - loss: 1.3164 - accuracy: 0.8477 - val_loss: 1.3302 - val_accuracy: 0.8288
Epoch 7/10
239/239 [==============================] - 88s 369ms/step - loss: 1.2973 - accuracy: 0.8673 - val_loss: 1.3320 - val_ac

In [ ]:
model_export_path = '/saved_model'
tf.saved_model.save(model, model_export_path)

### 5. Prediction and Evaluation 

In [16]:
def preprocess_input(params):
    csv_file = params['csv_file']
    base_dir = params['base_dir']
    
    builder = DatasetBuilder(base_dir, csv_file)
    _, _, test_df = builder.split_df()
    image_list = test_df.image_path.values
    
    image = []
    for img in image_list:
        img2array, _ = builder.decode_image(img)
        img_batch = np.expand_dims(img2array, axis=0)
        image.append(img_batch)
    
    labels = builder.get_labels(test_df)
    images = np.vstack(image)
    return images, labels

In [17]:
test_images, test_labels = preprocess_input(params)

In [18]:
predictions = model.predict(test_images)

In [19]:
true_labels = pd.DataFrame(data = test_labels) 
predicted_labels = pd.DataFrame(data = predictions) 